In [ ]:
#needed dependencies
!pip install langchain
!pip install llama-cpp-python
!pip install transformers
!pip install pyllamacpp==2.1.1
!pip install chromadb

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Chroma

In [2]:
loader = TextLoader('doc.txt')

In [3]:
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=0)
docs = text_splitter.split_documents(docs)


In [4]:
db = Chroma.from_documents(docs)


[2023-05-02 23:20:38,164] {posthog.py:15} INFO - Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
[2023-05-02 23:20:38,169] {__init__.py:80} INFO - Running Chroma using direct local API.
[2023-05-02 23:20:39,265] {__init__.py:49} WARNING - Using embedded DuckDB without persistence: data will be transient
[2023-05-02 23:20:39,341] {ctypes.py:22} INFO - Successfully imported ClickHouse Connect C data optimizations
[2023-05-02 23:20:39,357] {ctypes.py:31} INFO - Successfully import ClickHouse Connect C/Numpy optimizations
[2023-05-02 23:20:39,371] {json_impl.py:45} INFO - Using python library for writing JSON byte strings
[2023-05-02 23:20:39,557] {Collection.py:52} WARNING - No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction
[2023-05-02 23:20:47,225] {SentenceTransformer.py:66} INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
[2023-05-02 23:20:49,544] {SentenceTransformer.py:105} I

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
query = "What is purpose of NATO"
ans = db.similarity_search(query,k=1)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
ans = ans[0].page_content

demos that have been released called Bach.


In [ ]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

#Download the model
hf_hub_download(repo_id="LLukas22/gpt4all-lora-quantized-ggjt", filename="ggjt-model.bin", local_dir=".")

#Load the model
model = Llama(model_path="ggjt-model.bin")

#Generate
prompt=f'''Please use the following context to answer questions.
Context: {ans}
---
Question: {query}
Answer:'''

result=model(prompt, max_tokens=100)
#Print result with additional infos
print(result)

#Get only the generated text
generated_text = result["choices"][0]["text"]
print(generated_text)